This Query is created by user sohamjcpp

In [2]:
%% sql
```
WITH pcom_customer_cte AS (
  SELECT 
    cust_code AS intermediary_code,
    cust_name AS intermediary_name,
    'replicate_logic_from_policy_master' AS intermediary_type,
    'replicate_logic_from_policy_master' AS agent_mobile_no,
    'replicate_logic_from_policy_master' AS agent_email_id 
  FROM Premia.Pcom_customer 
  WHERE SUBSTR(cust_code, 1, 2) IN (SELECT dim_intermediary_type FROM dim_intermediary_type)
),
t_rpdb_pol_party_cte AS (
  SELECT 
    cust_code AS intermediary_code,
    cust_name AS intermediary_name
  FROM Bancs.T_RPDB_POL_PARTY
),
agg_policy_master_report_daily_cte AS (
  SELECT
    fufiller_code,
    fulfiller_name,
    office_code,
    office_name,
    zone_name,
    mkt_zone_name,
    ROW_NUMBER() OVER (PARTITION BY intermediary_code ORDER BY policy_from_date DESC) AS rn
  FROM agg_policy_master_report_daily
)
SELECT
  pcom_customer_cte.intermediary_code,
  pcom_customer_cte.intermediary_name,
  pcom_customer_cte.intermediary_type,
  pcom_customer_cte.agent_mobile_no,
  pcom_customer_cte.agent_email_id,
  'Direct map' AS is_aml_acknowledged,
  'Direct map' AS aml_acknowledged_timestamp,
  CASE
    WHEN CURRENT_DATE - pcom_customer_cte.aml_acknowledged_timestamp >= 365 THEN 'Yes'
    ELSE 'No'
  END AS due_aml_training,
  CURRENT_DATE - INTERVAL '1 day' AS reported_date,
  CURRENT_TIMESTAMP AS load_timestamp,
  agg_policy_master_report_daily_cte.fufiller_code,
  agg_policy_master_report_daily_cte.fulfiller_name,
  agg_policy_master_report_daily_cte.office_code,
  agg_policy_master_report_daily_cte.office_name,
  agg_policy_master_report_daily_cte.zone_name,
  agg_policy_master_report_daily_cte.mkt_zone_name
FROM pcom_customer_cte
LEFT JOIN t_rpdb_pol_party_cte ON pcom_customer_cte.intermediary_code = t_rpdb_pol_party_cte.cust_code
LEFT JOIN agg_policy_master_report_daily_cte ON pcom_customer_cte.intermediary_code = agg_policy_master_report_daily_cte.intermediary_code
WHERE agg_policy_master_report_daily_cte.rn = 1;
```